In [1]:
import os
import numpy as np
import h5py
import dask.array as da

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1) Создать массив размерностью 1000 на n (n - кратно тысячам), заполненный нормально распределенными числами (матожидание 0.0, стандартное отклонение 1.0), и сохранить его в файле(ах) с расширением hdf5

In [2]:
n = 10
x = da.random.normal(0, 1, size=(1000,n), chunks=(100,100))

In [3]:
with h5py.File('Data.h5', 'w') as s:
    s.create_dataset('data_set_1', data=x)

Решить 2я способами: с помощью NumPy и с помощью Dask Array и сравнить время выполнения.

.2) Считать массив из файла и загрузить в dask array

In [4]:
f = h5py.File(os.path.join(os.getcwd(), 'Data.h5'), mode='r')
data_s = f['/data_set_1']

In [5]:
data_s

<HDF5 dataset "data_set_1": shape (1000, 10), type "<f8">

In [6]:
x = da.from_array(data_s, chunks=(100, 100))

In [7]:
x.compute()

array([[-1.51530061,  0.76893578, -1.28140785, ..., -0.63871826,
        -1.84481996,  1.32040641],
       [ 1.01983646,  2.50369093, -0.94165615, ...,  0.29316609,
        -0.4467386 ,  0.11968536],
       [ 1.1304045 ,  1.11659027,  1.23057903, ..., -0.75956587,
         0.21384378, -0.06657034],
       ...,
       [-0.17001183, -1.81565067,  0.46436369, ..., -0.02920385,
        -0.83934452, -0.34783562],
       [ 0.72880238,  1.40056649, -0.03099372, ...,  0.71626331,
        -0.00285039, -0.91830854],
       [ 0.92533282, -1.10252013, -0.46856575, ..., -1.26337667,
        -1.30822275,  1.30377933]])

2.1) Найти и вывести на экран наименьший элемент в массиве.

In [8]:
%%time
x = x.persist()

Wall time: 6.95 ms


In [9]:
x.min().compute()

-3.7069721225923877

2.2) Найти и вывести на экран вектор-столбец из массива, содержащий наименьший элемент. И найти наименьший элемент в найденном столбце.

In [10]:
x.shape

(1000, 10)

In [11]:
#минимальные элементы по стоблцам
x.min(axis = 0).argmin().compute()

4

In [12]:
#столбец с минимальным элементом
x[:,x.min(axis = 0).argmin()].compute()

array([-1.77374952e-01,  1.24742484e+00, -9.19692002e-01, -7.63653145e-02,
       -1.66649243e+00,  1.70172276e+00, -1.61866696e+00, -1.04150062e+00,
       -4.99965615e-02,  3.80737725e-01, -1.71054473e+00, -3.07134209e-01,
        7.02564032e-01, -1.60261671e-01,  6.41067520e-01,  1.64907906e-03,
       -1.92891784e+00, -2.78299898e-01, -1.13026100e+00,  7.54985051e-01,
       -9.00750363e-01,  1.07501105e+00, -2.30640048e+00, -1.37251500e+00,
       -4.06055964e-01,  8.65897525e-01, -1.29041502e+00, -1.92824536e+00,
        1.42709915e+00, -1.14522986e+00,  8.24782207e-01,  1.41228320e-01,
        2.27078144e+00, -5.41522623e-01,  7.77214868e-01,  2.16464164e+00,
        1.22804652e+00,  1.62936456e+00, -7.36941063e-01, -5.06137304e-01,
        5.50983056e-01, -1.48218838e+00, -8.19471368e-01, -2.41126461e-01,
        1.42608117e+00, -1.32325784e+00, -1.05564898e+00,  2.15734286e-01,
        1.28360277e+00, -2.62429023e+00,  7.60989809e-01, -7.67221898e-03,
        4.43617130e-02, -

3) Сгенирировать нормально распределенный одномерный массив из 1000 элементов (вектор $a$). Найти ближайший к нему (в смысле L2) вектор-столбец из исходного массива (вектор $v_n$).

In [13]:
x_3 = da.random.normal(0, 1, size=(1, 1000), chunks=(1, 100))

In [14]:
x_3.shape

(1, 1000)

In [15]:
da.linalg.norm(x_3).compute()

31.44406367653344

In [16]:
da.linalg.norm(x, axis=0).compute()

array([32.37017502, 32.07924678, 32.3431391 , 30.88980883, 31.01467588,
       31.40739203, 30.57903962, 32.89455482, 31.3691016 , 32.47810772])

In [17]:
get_norms = da.linalg.norm(x_3).compute() - da.linalg.norm(x, axis=0).compute()

In [18]:
get_norms

array([-0.92611134, -0.63518311, -0.89907543,  0.55425485,  0.42938779,
        0.03667165,  0.86502406, -1.45049115,  0.07496208, -1.03404404])

In [19]:
np.abs(get_norms - 0).argmin()

5

In [20]:
get_norms[np.abs(get_norms - 0).argmin()]

0.03667164593579031

In [21]:
x[:, np.abs(get_norms - 0).argmin()].compute()

array([ 2.57594016e-01,  1.63135342e-01, -1.28751463e+00,  3.29933281e-01,
       -4.13885198e-01, -2.94323677e-01, -2.07010183e+00,  8.97078468e-01,
       -1.07907587e+00, -5.44478392e-01,  1.20451413e+00,  1.45740807e+00,
        2.01341243e+00, -9.54548899e-01, -1.62205204e-01,  1.86935428e+00,
       -5.78401079e-01, -1.46377223e-01,  1.38322758e-01, -1.06523675e-01,
       -4.32907128e-01, -8.09686324e-01,  3.58434482e-01,  2.44939635e-01,
        1.34183783e+00, -1.58485684e+00, -1.12859102e+00,  4.01633609e-01,
       -2.73074072e-01,  1.01969864e+00,  1.17151294e+00,  1.59386785e+00,
       -8.35422515e-01,  1.29412406e+00, -1.05099244e-01,  5.76048520e-01,
        8.13720449e-03, -1.38945661e-01,  8.37018525e-01,  5.67508493e-01,
        1.15926393e-02,  5.74157829e-01, -5.16153966e-01, -6.25821187e-01,
        1.52430906e-01,  8.00194144e-01,  1.01667613e+00,  1.87934125e+00,
        7.51167877e-01,  1.91709687e+00, -2.05668454e-01, -8.32183855e-01,
       -1.04219015e+00,  

In [32]:
len_x = da.sqrt(da.sum(da.power(x_3, 2)))
lens_arr = da.sqrt(da.sum(da.power(x, 2), axis=0))
res = x[:, da.absolute(lens_arr - len_x).argmin()].reshape(-1, 1).compute()

In [33]:
res

array([[ 2.57594016e-01],
       [ 1.63135342e-01],
       [-1.28751463e+00],
       [ 3.29933281e-01],
       [-4.13885198e-01],
       [-2.94323677e-01],
       [-2.07010183e+00],
       [ 8.97078468e-01],
       [-1.07907587e+00],
       [-5.44478392e-01],
       [ 1.20451413e+00],
       [ 1.45740807e+00],
       [ 2.01341243e+00],
       [-9.54548899e-01],
       [-1.62205204e-01],
       [ 1.86935428e+00],
       [-5.78401079e-01],
       [-1.46377223e-01],
       [ 1.38322758e-01],
       [-1.06523675e-01],
       [-4.32907128e-01],
       [-8.09686324e-01],
       [ 3.58434482e-01],
       [ 2.44939635e-01],
       [ 1.34183783e+00],
       [-1.58485684e+00],
       [-1.12859102e+00],
       [ 4.01633609e-01],
       [-2.73074072e-01],
       [ 1.01969864e+00],
       [ 1.17151294e+00],
       [ 1.59386785e+00],
       [-8.35422515e-01],
       [ 1.29412406e+00],
       [-1.05099244e-01],
       [ 5.76048520e-01],
       [ 8.13720449e-03],
       [-1.38945661e-01],
       [ 8.3

4) Определить сколько векторов-столбцов из исходного массива находятся на расстоянии, не превышающем $3|a-v_n|_{L2}$

In [49]:
x_3 = da.random.normal(0, 1, size=(1, 1000), chunks=(1, 100))

In [66]:
len_x = da.sqrt(da.sum(da.power(x_3, 2)))
lens_arr = da.sqrt(da.sum(da.power(x, 2), axis=0))
dists = da.absolute(lens_arr - len_x)
dist = 3 * dists.min()
mask = dists <= dist

In [67]:
mask.compute()

array([ True,  True,  True, False, False, False, False, False, False,
       False])

In [68]:
x_3[:, mask].compute()

ValueError: Chunks do not add up to shape. Got chunks=((100, 100, 100, 100, 100, 100, 100, 100, 100, 100),), shape=(10,)